In [3]:
from __future__ import division, print_function
import numpy as np
from ase.build import bulk
from ase.calculators.lj import LennardJones
from ase.constraints import UnitCellFilter
from ase.optimize import MDMin

# Theoretical infinite-cutoff LJ FCC unit cell parameters
vol0 = 4 * 0.91615977036  # theoretical minimum
a0 = vol0**(1 / 3)

a = bulk('Al', 'fcc', a=a0)
cell0 = a.get_cell()

a.calc = LennardJones()
a.set_cell(np.dot(a.cell,
                  [[1.02, 0, 0.03],
                   [0, 0.99, -0.02],
                   [0.1, -0.01, 1.03]]),
           scale_atoms=True)

a *= (1, 2, 3)
cell0 *= np.array([1, 2, 3])[:, np.newaxis]

a.rattle()

# Verify analytical stress tensor against numerical value
s_analytical = a.get_stress()
s_numerical = a.calc.calculate_numerical_stress(a, 1e-5)
s_p_err = 100 * (s_numerical - s_analytical) / s_numerical

print("Analytical stress:\n", s_analytical)
print("Numerical stress:\n", s_numerical)
print("Percent error in stress:\n", s_p_err)
assert np.all(abs(s_p_err) < 1e-5)

# Minimize unit cell
opt = MDMin(UnitCellFilter(a), dt=0.01)
opt.run(fmax=1e-3)

# Verify minimized unit cell using Niggli tensors
g_minimized = np.dot(a.cell, a.cell.T)
g_theory = np.dot(cell0, cell0.T)
g_p_err = 100 * (g_minimized - g_theory) / g_theory

print("Minimized Niggli tensor:\n", g_minimized)
print("Theoretical Niggli tensor:\n", g_theory)
print("Percent error in Niggli tensor:\n", g_p_err)
assert np.all(abs(g_p_err) < 1)

Analytical stress:
 [-0.77895227  1.2288922  -1.22307094 -1.37861825  5.74494294 -0.2421896 ]
Numerical stress:
 [-0.77895229  1.22889219 -1.22307096 -1.37861826  5.74494297 -0.2421896 ]
Percent error in stress:
 [ 1.77660153e-06 -9.54663186e-07  1.38792921e-06  5.22451828e-07
  5.11184747e-07  3.71703910e-08]
       Step     Time          Energy         fmax
MDMin:    0 22:02:36      -45.377826        5.7316
MDMin:    1 22:02:36      -45.381235        5.7242
MDMin:    2 22:02:36      -45.391404        5.7023
MDMin:    3 22:02:36      -45.408165        5.6662
MDMin:    4 22:02:36      -45.431288        5.6165
MDMin:    5 22:02:36      -45.460511        5.5538
MDMin:    6 22:02:36      -45.495563        5.4790
MDMin:    7 22:02:37      -45.536141        5.3931
MDMin:    8 22:02:37      -45.581909        5.2971
MDMin:    9 22:02:37      -45.632491        5.1923
MDMin:   10 22:02:37      -45.687487        5.0797
MDMin:   11 22:02:37      -45.746477        4.9607
MDMin:   12 22:02:37      

In [2]:
from math import sqrt
from ase import Atoms
from ase.constraints import StrainFilter
from ase.optimize.mdmin import MDMin
from ase.io import Trajectory
try:
    from asap3 import EMT
except ImportError:
    pass
else:
    a = 3.6
    b = a / 2
    cu = Atoms('Cu', cell=[(0,b,b),(b,0,b),(b,b,0)], pbc=1) * (6, 6, 6)

    cu.set_calculator(EMT())
    f = StrainFilter(cu, [1, 1, 1, 0, 0, 0])
    opt = MDMin(f, dt=0.01)
    t = Trajectory('Cu.traj', 'w', cu)
    opt.attach(t)
    opt.run(0.001)

# HCP:
    from ase.build import hcp0001
    cu = hcp0001('Cu', (1, 1, 2), a=a / sqrt(2))
    cu.cell[1,0] += 0.05
    cu *= (6, 6, 3)

    cu.set_calculator(EMT())
    f = StrainFilter(cu)
    opt = MDMin(f, dt=0.01)
    t = Trajectory('Cu.traj', 'w', cu)
    opt.attach(t)
    opt.run(0.01)


In [2]:
import numpy as np

from ase import Atoms
from ase.io.trajectory import Trajectory
from ase.calculators.emt import EMT

a = 4.0  # approximate lattice constant
b = a / 2
ag = Atoms('Ag',
           cell=[(0, b, b), (b, 0, b), (b, b, 0)],
           pbc=1,
           calculator=EMT())  # use EMT potential
cell = ag.get_cell()
traj = Trajectory('Ag.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    ag.set_cell(cell * x, scale_atoms=True)
    ag.get_potential_energy()
    traj.write(ag)

In [3]:
from ase.io import read
from ase.units import kJ
from ase.eos import EquationOfState
configs = read('Ag.traj@0:5')  # read 5 configurations
# Extract volumes and energies:
volumes = [ag.get_volume() for ag in configs]
energies = [ag.get_potential_energy() for ag in configs]
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
print(B / kJ * 1.0e24, 'GPa')
eos.plot('Ag-eos.png')

100.14189241975146 GPa
